# Data analysis of Oxford Parkinson's Disease Detection Dataset

## Student: Filomeno Davide Miro
## Matricola: 256870
## Teacher: Prof.Francesco Vaccarino

# Table of Contents

- [Introduction](#Introduction)
- [Data visualization](#Data-visualization)
- [Data exploration](#Data-exploration)
- [Features extraction](#Features-extraction)
- [Classification](#Classification)
    - [Logistic Regression](#Logistic-Regression)
    - [K Nearest Neighbours](#K-Nearest-Neighbours)
    - [Support Vector Machines](#Support-Vector-Machines)
    - [Decision tree](#Decision-tree)
    - [Random forest](#Random-forest)
- [Conclusion](#Conclusion)

# Introduction

The dataset contains 195 biomedical voice measurements taken from 31 individuals,23 of them with Parkinson’s disease.Each line corresponds to a voice measurement taken by a patient.The aim of this analysis is to discriminate patients with Parkinson’s disease (labelled with 0) with patients without the disease (labelled with 1) by using the 23 attributes.The attribute information are the same:
- **name** - ASCII subject name and recording number
- **MDVP:Fo(Hz)** - Average vocal fundamental frequency
- **MDVP:Fhi(Hz)** - Maximum vocal fundamental frequency
- **MDVP:Flo(Hz)** - Minimum vocal fundamental frequency
- **MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP** - Several measures of variation in fundamental frequency
- **MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,MDVP:APQ,Shi mmer:DDA** - Several measures of variation in amplitude
- **NHR,HNR** - Two measures of ratio of noise to tonal components in the voice RPDE,D2 - Two nonlinear dynamical complexity measures
- **DFA** - Signal fractal scaling exponent
- **spread1,spread2,PPE** - Three nonlinear measures of fundamental frequency variation status - Health status of the subject (one) - Parkinson's, (zero) - healthy.It is the target class.

# Data Visualization

In [926]:
#pandas and numpy
import pandas as pd
import numpy as np

#Preprocessing
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

#ML algorithm
from sklearn.decomposition import PCA
import sklearn.neighbors as knn
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


#SMOTE
from imblearn.pipeline import make_pipeline, Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

#Metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import plot_confusion_matrix


#Model selection
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GridSearchCV, learning_curve


#plot libaries
import plotly
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True) # to show plots in notebook

# online plotly
#from chart_studio.plotly.plotly import plot, iplot
#plotly.tools.set_credentials_file(username='davidemiro', api_key='Z3wizWCFUj4wCd5ZmRcn')

# offline plotly
from plotly.offline import plot, iplot



# read the data
data = pd.read_csv("parkinsons.csv")
print("The dataset has %d rows and %d columns." % data.shape)

# check for null values in the dataset
print("There are " + ("some" if data.isnull().values.any() else "no")  + " null/missing values in the dataset.")

SEED = 9 # specify seed for reproducable results
pd.set_option('display.max_columns', None) # prevents abbreviation (with '...') of columns in prints

The dataset has 195 rows and 24 columns.
There are no null/missing values in the dataset.


We first look at some rows of the dataset by using the *head()* function.

In [927]:
data.head()

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,MDVP:APQ,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,0.426,0.02182,0.03130,0.02971,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,0.626,0.03134,0.04518,0.04368,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,0.482,0.02757,0.03858,0.03590,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,0.517,0.02924,0.04005,0.03772,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,0.584,0.03490,0.04825,0.04465,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


We can se that the column **name** is an identifier of the sample and so is useless for this analisis.
The column **status** is the target class that we want to predict.It is 1 for samples coming from health people and 0 for people with parkinson disease.
The goal of this analysis is to understand how to predict whether a person has the disease or not and this assigment of the target class can be problematic for metric computation.The next action is to reassign the values of the status attribute to have the value 1 if the person the sample came from has the disease, 0 otherwise.

In [928]:
def adjustData(df):
    new_df = df.copy()
    # convert string columns to integers
    new_df["status"] = new_df["status"].apply(lambda x: 0 if x==1 else 1)
    new_df = new_df.drop(["name"], axis=1)
    return new_df

new_data = adjustData(data)

##### Statistical overview of the data

The *describe* function of DataFrame class provides some statistical measures of all the columns of the data.

- count: number of samples
- mean: the mean of this attribute among all samples
- std: the standard deviation of this attribute
- min: the minimal value of this attribute
- 25%: the lower percentile
- 50%: the median
- 75%: the upper percentile
- max: the maximal value of this attribute

In [929]:
new_data.describe()

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,MDVP:APQ,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
count,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000
mean,154.228641,197.104918,116.324631,0.006220,0.000044,0.003306,0.003446,0.009920,0.029709,0.282251,0.015664,0.017878,0.024081,0.046993,0.024847,21.885974,0.246154,0.498536,0.718099,-5.684397,0.226510,2.381826,0.206552
std,41.390065,91.491548,43.521413,0.004848,0.000035,0.002968,0.002759,0.008903,0.018857,0.194877,0.010153,0.012024,0.016947,0.030459,0.040418,4.425764,0.431878,0.103942,0.055336,1.090208,0.083406,0.382799,0.090119
min,88.333000,102.145000,65.476000,0.001680,0.000007,0.000680,0.000920,0.002040,0.009540,0.085000,0.004550,0.005700,0.007190,0.013640,0.000650,8.441000,0.000000,0.256570,0.574282,-7.964984,0.006274,1.423287,0.044539
25%,117.572000,134.862500,84.291000,0.003460,0.000020,0.001660,0.001860,0.004985,0.016505,0.148500,0.008245,0.009580,0.013080,0.024735,0.005925,19.198000,0.000000,0.421306,0.674758,-6.450096,0.174351,2.099125,0.137451
50%,148.790000,175.829000,104.315000,0.004940,0.000030,0.002500,0.002690,0.007490,0.022970,0.221000,0.012790,0.013470,0.018260,0.038360,0.011660,22.085000,0.000000,0.495954,0.722254,-5.720868,0.218885,2.361532,0.194052
75%,182.769000,224.205500,140.018500,0.007365,0.000060,0.003835,0.003955,0.011505,0.037885,0.350000,0.020265,0.022380,0.029400,0.060795,0.025640,25.075500,0.000000,0.587562,0.761881,-5.046192,0.279234,2.636456,0.252980
max,260.105000,592.030000,239.170000,0.033160,0.000260,0.021440,0.019580,0.064330,0.119080,1.302000,0.056470,0.079400,0.137780,0.169420,0.314820,33.047000,1.000000,0.685151,0.825288,-2.434031,0.450493,3.671155,0.527367


In [930]:
colors = plotly.colors.DEFAULT_PLOTLY_COLORS
label_dict = {0: "No disease", 1: "Parkinson"}

y = new_data["status"].value_counts()

data_chart = [go.Bar(x=[label_dict[x] for x in y.index], y=y.values, marker = dict(color = colors[:len(y.index)]))]
layout = go.Layout(
    title='Class label distribution',
    autosize=False,
    width=400,
    height=400,
    yaxis=dict(
        title='#samples',
    ),
)
fig = go.Figure(data=data_chart, layout=layout)
iplot(fig, filename='basic-bar3')

In [931]:
parkinson_perc = new_data["status"].sum() * 100 / new_data["status"].shape[0]
print("Parkinson percentage is %.3f%%." % parkinson_perc)

Parkinson percentage is 24.615%.


There are more samples of class *No disease* than the class *Parkinson*. 
The next graphs show the distributions of all the features of the data.
All the features are numerical.
The **histogram** is able to visualize the distribution of the values of a numeric feature in a way that allows to visualize for which values of that feature the sample belongs to that or that other class.
The **box plot** is a visualization tool focused on the visualization of statistical measures on the distribution of values.
These statistical measures are:

- median
- first quartile (Q1): the middle number between the  minimum value and the median of the distribution.
- third quartile (Q3): the middle number between the maximum value and the median of the distribution.
- interquartile range (IQR): the difference between Q3 and Q1.It's represented as a block that encloses the 50% of the values.
- lower fence (Q1 - 1.5 IQR) and the upper fence (Q3 + 1.5 IQR).
- maximum and the minimum value.
- outliers: data objects that deviate significantly from the rest of the objects, are extreme values.

In [932]:
Parkinson = new_data[new_data["status"] == 1]
no_Parkinson= new_data[new_data["status"] == 0]

In [933]:
def create_Parkinson_trace(col, visible=False):
    return go.Histogram(
        x=Parkinson[col],
        name='Parkinson',
        marker = dict(color = colors[1]),
        visible=visible,
    )

def create_no_Parkinson_trace(col, visible=False):
    return go.Histogram(
        x=no_Parkinson[col],
        name='no Parkinson',
        marker = dict(color = colors[0]),
        visible = visible,
    )


features_for_hist = [x for x in new_data.columns if x not in ["status"]]
active_idx = 0
traces_Parkinson = [(create_Parkinson_trace(col) if i != active_idx else create_Parkinson_trace(col, visible=True)) for i, col in enumerate(features_for_hist)]
traces_no_Parkinson = [(create_no_Parkinson_trace(col) if i != active_idx else create_no_Parkinson_trace(col, visible=True)) for i, col in enumerate(features_for_hist)]
data_vis = traces_Parkinson + traces_no_Parkinson

n_features = len(features_for_hist)
steps = []
for i in range(n_features):
    step = dict(
        method = 'restyle',
        args = ['visible', [False] * len(data_vis)],
        label = features_for_hist[i],
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    step['args'][1][i + n_features] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active = active_idx,
    currentvalue = dict(
        prefix = "Feature: ",
        xanchor= 'center',
    ),
    pad = {"t": 50},
    steps = steps,
)]

layout = dict(
    sliders=sliders,
    yaxis=dict(
        title='#samples',
        automargin=True,
    ),
)

fig = dict(data=data_vis, layout=layout)

iplot(fig, filename='histogram_slider')

After this visualization we can se that in most features, data labeled as positive get the lowest values.We can se in the boxplots below that the median value of positive examples is often lower than the median value of negative ones.



In [934]:
def create_box_Parkinson_trace(col, visible=False):
    return go.Box(
        y=Parkinson[col],
        name='Parkinson',
        marker = dict(color = colors[1]),
        visible=visible,
    )

def create_box_no_Parkinson_trace(col, visible=False):
    return go.Box(
        y=no_Parkinson[col],
        name='no Parkinson',
        marker = dict(color = colors[0]),
        visible = visible,
    )


features_for_hist = [x for x in new_data.columns if x not in ["status"]]
# remove features with too less distinct values (e.g. binary features), because boxplot does not make any sense for them
features_for_box = [col for col in features_for_hist if len(Parkinson[col].unique())>5]

active_idx = 0
box_traces_Parkinson = [(create_box_Parkinson_trace(col) if i != active_idx else create_box_Parkinson_trace(col, visible=True)) for i, col in enumerate(features_for_box)]
box_traces_no_Parkinson = [(create_box_no_Parkinson_trace(col) if i != active_idx else create_box_no_Parkinson_trace(col, visible=True)) for i, col in enumerate(features_for_box)]
data_vis = box_traces_Parkinson + box_traces_no_Parkinson

n_features = len(features_for_box)
steps = []
for i in range(n_features):
    step = dict(
        method = 'restyle',
        args = ['visible', [False] * len(data_vis)],
        label = features_for_box[i],
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    step['args'][1][i + n_features] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active = active_idx,
    currentvalue = dict(
        prefix = "Feature: ",
        xanchor= 'center',
    ),
    pad = {"t": 50},
    steps = steps,
    len=1,
)]

layout = dict(
    sliders=sliders,
    yaxis=dict(
        title='value',
        automargin=True,
    ),
    legend=dict(
        x=0,
        y=1,
    ),
)

fig = dict(data=data_vis, layout=layout)

iplot(fig, filename='box_slider')

The boxplots depict also some outliers in the features.We can have different kind of outliers considering their nature: point outliers, contextual outliers, or collective outliers. Point outliers are single data points that are far from the rest of the distribution; contextual outliers can be noise in data, such as background noise signal when doing speech recognition; collective outliers can be subsets of novelties in data such as a signal that may indicate the discovery of new phenomena.In this case we can lead back the nature of the outliers to the first kind and so I decide to keep them because they can potentially keep a lot of informations.

# Data exploration

In this section we want to investigate about potentially correlations between the features. We do that by using some mathematical tools like **Pearson correlation**.
Given two variables $ \mathit{X} $ , $ \mathit{Y} $ and theirs standard deviations $ \sigma_{\mathit{X}} $,$ \sigma_{\mathit{Y}} $ the covariance is $ \mathrm{cov} (\mathit{X},\mathit{Y} ) = \mathit{E}[(\mathit{X}-\mathit{E}[\mathit{X}])(\mathit{Y}-\mathit{E}[\mathit{Y}])] $.
The **Pearson correlation** is defined as: $\rho_{\mathit{X},\mathit{Y}}= \frac{\mathrm{cov} (\mathit{X},\mathit{Y} )}{\rho_{\mathit{X}}\rho_{\mathit{Y}}}$.
To visualize the Pearson correlation score of each pair of features is plotted below the correlation matrix.
An high correlations are coloured more to the yellow and lower ones more to the blue.

In [935]:
corr = new_data.corr()
trace = go.Heatmap(z=corr.abs().values.tolist(), x=corr.columns, y=corr.columns)
data_vis=[trace]
layout = go.Layout(
    title='Heatmap of pairwise correlation of the columns',
    autosize=False,
    width=850,
    height=700,
    yaxis=go.layout.YAxis(automargin=True),
    xaxis=dict(tickangle=40),
    margin=go.layout.Margin(l=0, r=200, b=200, t=80)
)


fig = go.Figure(data=data_vis, layout=layout)
iplot(fig, filename='labelled-heatmap1')


We can observe that among all the features there is on average a high correlation score. Furthermore, by comparing these results with the information on the nature of the features, it can be seen that groups of features with similar technical characteristics (MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP) and (MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,MDVP:APQ,Shimmer:DDA) are strongly correlated with each other.
Given this situation,we can filter some features that are strongly correlated with other ones and not provide usefull informations.
We can help this step of features selection by performing an agglomerative clustering over the features.This algorithm starts by assigning a cluster to each feature and at each step  compute a pairwise similarity between all the clusters.In this case the similarity measure is the correlation.Then are selected the clusters with the higher average correlation and  merged toghether.This process is repeated until all the features are merged to one cluster.
The result of this clustering technique is illustrate by the dendogram below.


In [936]:
from scipy.cluster import hierarchy as hc
X = np.random.rand(10, 10)
names = new_data.columns
inverse_correlation = 1 - abs(new_data.corr())
fig = ff.create_dendrogram(inverse_correlation.values, orientation='left', labels=names, colorscale=colors, linkagefun=lambda x: hc.linkage(x, 'average'))
fig['layout'].update(dict(
    title="Dendrogram of clustering the features according to correlation",
    width=800,
    height=600,
    margin=go.layout.Margin(l=180, r=50),
    xaxis=dict(
        title='distance',
    ),
    yaxis=dict(
        title='features',
        automargin=True,
    ),
))
iplot(fig, filename='dendrogram_corr_clustering')

We can se that the following features are very close:

- spread 1, PPE
- Shimmer APQ3,Shimmer:DDA,MVDP:Shimmer,MVDP:Shimmer(dB),Shimmer APQ5,MVDP:APQ
- MVDP:PPQ,MVDP:Jitter(%),MVDP:RAP,Jitter DDP.

Given these three clusters of features we can decide to keep only one element, without significantly losing information.
The features PPE,Shimmer APQ3,Shimmer:DDA,MVDP:Shimmer(dB),Shimmer APQ5,MVDP:APQ,MVDP:Jitter(%),MVDP:RAP and Jitter DDP are removed from the data.

In [937]:
close_features = ['PPE','Shimmer:APQ3','Shimmer:DDA','MDVP:Shimmer(dB)','Shimmer:APQ5','MDVP:APQ','MDVP:Jitter(%)','MDVP:RAP','Jitter:DDP']

#Extract the class label from the dataset
y = new_data["status"]
print(new_data.columns)
#Preprocessing for knn,svm,lg
X = new_data.drop(["status"]+close_features, axis=1)
X_normed = (X - X.mean()) / X.std()

#Preprocessing for decision tree,random forest
X_tree = new_data.drop(["status"], axis=1)
X_tree = (X_tree - X_tree.mean()) / X_tree.std()

Index(['MDVP:Fo(Hz)', 'MDVP:Fhi(Hz)', 'MDVP:Flo(Hz)', 'MDVP:Jitter(%)',
       'MDVP:Jitter(Abs)', 'MDVP:RAP', 'MDVP:PPQ', 'Jitter:DDP',
       'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5',
       'MDVP:APQ', 'Shimmer:DDA', 'NHR', 'HNR', 'status', 'RPDE', 'DFA',
       'spread1', 'spread2', 'D2', 'PPE'],
      dtype='object')


# Features Extraction

In this section we want to reduce the dimension of our dataset by using the **Principal Component Analysis**.
It would be convenient to reduce the number of data dimensions to be able to visualize them together and to avoid the problem of “curse of dimensionality” that affect machine learning algorithms like k-nearest neighbours.**Principal Component Analysis** is an unsupervised algorithm that ,starting from input data, is able to project them in a lower dimensional space that maximize the information kept.


Given you samples $ x_{1},...,x_{n} \in \mathbb{R}^{D} $ the goal is to project them onto a space having dimensionality $ d < D $ while maximizing the variance of the projected data.We can define the directions of this space using a set of $D$-dimensional vectors $u_{i}$ that are orthonormal ($\langle u_{i},u_{j} \rangle = 0 $ and $ \Vert u_{i} \Vert = 1 $. These vectors are called **principal components**.

The variance associated to a principal component is equal to 

$ \mathit{Var(u_{i}) = u^{T}\Sigma u } $ 

where $ \Sigma = \frac{1}{N}\sum^{N}_{n=1}\left(x_{n} - \overline{x}\right)\left(x_{n} - \overline{x}\right) $ is the correlation matrix of your data with mean $ \overline{x} = \frac{1}{N}\sum^{N}_{n=1} x_{n} $.

The definition of the problem is the following:

$ \mathit{u_{1} = argmax_{\Vert u_{1} \Vert = 1} Var(u_{1}) = argmax \ u^{T} \Sigma u} 
. \\
. \\
. \\
 \mathit{u_{d} =argmax_{\Vert u_{d} \Vert = 1 \ and \ \langle u_{d},u_{i} \rangle = 0 \ \forall \ i <  d }  \ Var(u_{d}) = argmax \ u^{T} \Sigma u} $

It is possible to demostrate that the solutions of this problem are the right singular vector of the **singular value decomposition** of the matrix $ X_{c} = \frac{1}{N}\sum^{N}_{n=1}\left(x_{n} - \overline{x}\right) $.
The variance associated with each right singular vector is given by the  corresponding singular value. Therefore, to have the components with maximum variance, these are sorted in decreasing order with respect to the value of the single value.This is  what the implementation of the algorithm provided by the sklearn library does. Before run PCA algorithm is important to scale the data (mean = 0 and variance = 1),the tool that do that is StandardScaler().


In [938]:
#Re-scale before applying PCA
X_scaled = StandardScaler().fit_transform(X_normed)
pca = PCA(random_state=SEED)
X_pca = pca.fit_transform(X_scaled)

The following graph is a box plot that represents the percentage of variance explained by each principal component.
The scatter plot over the box plot visualizes the cumulative variance explained by the principal components.In this way we have a visual tool that allow us to select the right number of components considering the total variance that they keep.

In [939]:
tot = sum(pca.explained_variance_)
var_exp = [(p / tot) * 100 for p in sorted(pca.explained_variance_, reverse=True)]
cum_var_exp = np.cumsum(var_exp)


trace_cum_var_exp = go.Bar(
    x=list(range(1, len(cum_var_exp) + 1)),
    y=var_exp,
    name="individual explained variance",
)
trace_ind_var_exp = go.Scatter(
    x=list(range(1, len(cum_var_exp) + 1)),
    y=cum_var_exp,
    mode='lines+markers',
    name="cumulative explained variance",
    line=dict(
        shape='hv',
    ))
data = [trace_cum_var_exp, trace_ind_var_exp]
layout = go.Layout(
    title='Individual and Cumulative Explained Variance',
    autosize=True,
    yaxis=dict(
        title='% of explained variance',
    ),
    xaxis=dict(
        title="principal components",
        dtick=1,
    ),
    legend=dict(
        x=1,
        y=1,
    ),
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='basic-bar')

Considering the PCA result on our dataset, we want to select the first 6 components that keep more than 90% of total variance(90.05%).
The following graphs are scatter plots that show the distributions of the data according to the different combinations of principal components. It can be seen that for pc1 the examples belonging to the positive class form homogeneous groups. This bodes well for the classification.
The other graph is called **biplot**.It has the function of representing the loadings and scores.The loadings are the principal components expressed using the starting features.The scores are the coordinates of the examples with respect to the principal components.

In [940]:
#BIPLOT
import plotly.express as px
n_components = 6
pca = PCA(n_components=n_components,random_state=SEED)

X_pca = pca.fit_transform(X_scaled)

loading = pca.components_.T * np.sqrt(pca.explained_variance_)

total_var = pca.explained_variance_ratio_.sum() * 100

labels = {str(i): f"PC {i+1}" for i in range(n_components)}


fig = px.scatter_matrix(
    X_pca,
    color=y,
    dimensions=range(n_components),
    labels=labels,
    title=f'Total Explained Variance: {total_var:.2f}%',
)
fig.update_traces(diagonal_visible=False)
fig.show()

loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

fig = px.scatter(X_pca, x=0, y=1, color=y)

for i, feature in enumerate(X_normed_fs.columns):
    fig.add_shape(
        type='line',
        x0=0, y0=0,
        x1=loadings[i, 0],
        y1=loadings[i, 1]
    )
    fig.add_annotation(
        x=loadings[i, 0],
        y=loadings[i, 1],
        ax=0, ay=0,
        xanchor="center",
        yanchor="bottom",
        text=feature,
    )
fig.show()

# Classification

In this section are illustrated the results of the training of different machine learning algorithms.
The algorithms that we will use are **Logistic Regression**,**K-Nearest Neighbours**,**Support Vector Machines**,**Decision Tree** and **Random Forest**.
Before starting to talk about the metrics used to evaluate the performance of our algorithm it is necessary to illustrate the difference between test error and training error.The **test error** is the average error that results from using a statistical learning method to predict the response on a new observation, one that was not used in training the method.In contrast, the **training error** can be easily calculated by applying the statistical learning method to the observations used in its training.Usually there is an importante difference between training and test error,and in particular the former can dramatically underestimate the latter.For this reason it is important to split the data available in two datasets: **training set** and **test set**.We use the first to train the algorithm and the last to calculate the metrics needed to evaluate the model's performance over data that model has never seen.
To have a reliable performance evaluation, the dataset is divided into two parts: training (70%) and testing (30%).The samples belonging to the testing dataset not participate to the training of the ML algorithms,they are used only to compute some metrics
to evaluate the model's ability to generalize to new data it has never used.


## Cross Validation

All the algorithms that have been used in this anaysis need to set one or more hyperparameters.We need to set hyperparameters that maximize the performance of our dataset in the test set.The **validation set approach** allow to enstimate the test error by taking a part of the training set and use it to evaluate the performances a trained model.The error estimate obtained with this approach strongly depends on which examples have been taken for validation and which for training.An improvement of this idea is given by **k-fold cross validation**.In this strategy the training set is splitted in k different sets and at each iteration one part is used to compute the predictions and the remaining K - 1 parts used to fit.This is done in turn for each part k = 1,2,...K, and then the results are combined. The error calculated by k-fold cross validation can be defined as :

$ CV_{(K)} = \sum_{k=1}^{K} \frac{n_{k}}{n}Error_{k} $

$ n $ is the number of examples in the training set, $ n_{k} $ is the number of examples of the $ k $ split.
If the value of $ K $ is too high (if $ K = n $ we have **leave one out CV**) the estimated error has an high variance.If the value of $ K $ is too small,the number of data that you will use to train your predictors 
becomes much smaller than the entire training set and the estimated error is characterized by a high bias. 
Values of $ k $ like 5 o 10 can be a good compromise for bias-variance trade-off. In this analysis is used a k - fold cross validation in a grid search to find the set of hyperparameters that get the minimum cross-validation error.In the function *StratifiedKFold* folds are made by preserving the percentage of samples for each class.The value of $ K $ is set to 5.





## Metrics

To measure the performance of the trained models were used some common metric.
Before showing it is good to know these quantities:
- **TP**: #correct predictions of positive class
- **TN**: #correct predictions of negative class
- **FP**: #wrong predictions of positive class
- **FN**: #wrong predictions of negative class

The used metrics are:
- **Accuracy**:  measures the proportion of correct predictions over the whole predictions.
 It’s expression is:

 $ \mathrm{accuracy} = \frac{TP + TN}{TP + TN +FP + FN} $

- **precision**: indicates the percentage of predictions that are relevant.

 $ \mathrm{precision} = \frac{TP }{TP  + FP }$

- **recall**: indicates the percentage of total relevant results correctly classified by the model.

 $ \mathrm{recall} = \frac{TP }{TP + FN}$

- **f1- score**: is the  harmonic mean of precision and recall.

 $ \mathrm{f1-score}=2  \frac{\mathrm{precision} \times \mathrm{recall}}{\mathrm{precision} + \mathrm{recall}} $

**Accuracy** can be problematic in datasets characterized by class imbalance problems.You can have an high accuracy if the model predicts the least representative class well even if in most cases it is wrong to predict the least representative class.In order to better measure the error of the models in cross validation and find parameters that allow the algorithms to achieve good performance in the positive class, in cross validation was used **f1-score**.

Other usefull visual representations are:

- **ROC curve**:is a plot that represents the variance of true positive rate (y-axis) and false positive rate (x-axis)considering different thresholds values.Small values in x-axis and high values in y-axis means that the fpr is low and instead the tpr is high.The perfect model is which that reach the point (0,1).You can measure how your model is close from this point by computing the Area Under the Curve (AUC metric).

    - $ \mathrm{True \ \ Positive \ \ Rate} = \frac{TP }{TP + FP}$

    - $ \mathrm{False \ \ Positive \ \ Rate} = \frac{FP }{FP + TN}$

- **Confusion matrix**: is a graphical representation of the quantities **TP**,**TN**,**FP**,**FN**.

- **Learning curve**: explains how the test and training error change by increasing the number of example in the training set. It is important to visualize that to avoid the phenomena of overfitting and understand if more data are needed (underfitting).

In [941]:
def get_color_with_opacity(color, opacity):
    return "rgba(" + color[4:-1] + ", %.2f)" % opacity

# partially based on https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html
def plot_learning_curve(estimator, title, X, y, cv=None, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5),visible=False):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """

    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=5, n_jobs=n_jobs, train_sizes=train_sizes, scoring="f1", random_state=SEED)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    trace1 = go.Scatter(
        x=train_sizes,
        y=train_scores_mean - train_scores_std,
        showlegend=False,
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = get_color_with_opacity(colors[0], 0.4),
        ),
        visible=visible,
    )
    trace2 = go.Scatter(
        x=train_sizes,
        y=train_scores_mean + train_scores_std,
        showlegend=False,
        fill="tonexty",
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = get_color_with_opacity(colors[0], 0.4),
        ),
        visible=visible,
    )
    trace3 = go.Scatter(
        x=train_sizes,
        y=train_scores_mean,
        showlegend=True,
        name="Train score",
        line = dict(
            color = colors[0],
        ),
        visible=visible,
    )

    trace4 = go.Scatter(
        x=train_sizes,
        y=test_scores_mean - test_scores_std,
        showlegend=False,
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = get_color_with_opacity(colors[1], 0.4),
        ),
        visible=visible,
    )
    trace5 = go.Scatter(
        x=train_sizes,
        y=test_scores_mean + test_scores_std,
        showlegend=False,
        fill="tonexty",
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = get_color_with_opacity(colors[1], 0.4),
        ),
        visible=visible,
    )
    trace6 = go.Scatter(
        x=train_sizes,
        y=test_scores_mean,
        showlegend=True,
        name="Test score",
        line = dict(
            color = colors[1],
        ),
        visible=visible,
    )

    data_vis = [trace1, trace2, trace3, trace4, trace5, trace6]
    '''
    layout = go.Layout(
        title=title,
        autosize=True,
        yaxis=dict(
            title='F1 Score',
        ),
        xaxis=dict(
            title="#Training samples",
        ),
        legend=dict(
            x=0.8,
            y=0,
        ),
    )
    fig = go.Figure(data=data_vis, layout=layout)
    '''
    return data_vis

def plot_two_feature_importance(features_importance_tree,features_importance_tree_sm,step_name):
    

    x = [[f[0] for f in features_importance_tree],[f[0] for f in features_importance_tree_sm]]
    y = [[f[1] for f in features_importance_tree],[f[1] for f in features_importance_tree_sm]]
    # Create figure
    fig = go.Figure()

    # Add traces, one for each slider step
    for i in range(2):
        fig.add_trace(
            go.Bar(
                visible=False,
                name="bo",
                x=x[i],
                y=y[i]))

    # Make 10th trace visible
    fig.data[0].visible = True



    # Create and add slider
    steps = []
    for i in range(len(fig.data)):
        step = dict(
            method="update",
            args=[{"visible": [False] * len(fig.data)},
                  {"title": "Features importance of  " + step_name[i]}],  # layout attribute
            label =step_name[i],
        )
        step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
        steps.append(step)

    sliders = [dict(
        active=0,
        currentvalue={"prefix": ""},
        pad={"t": 50},
        steps=steps
    )]

    fig.update_layout(
        sliders=sliders
    )

    fig.show()
def plot_feature_importance(feature_importance, title):
    trace1 = go.Bar(
        x=feature_importance[:, 0],
        y=feature_importance[:, 1],
        marker = dict(color = colors[0]),
        name='feature importance'
    )
    data = [trace1]
    layout = go.Layout(
        title=title,
        autosize=True,
        margin=go.layout.Margin(l=50, r=100, b=150),
        xaxis=dict(
            title='feature',
            tickangle=30
        ),
        yaxis=dict(
            title='feature importance',
            automargin=True,
        ),
    )
    fig = go.Figure(data=data, layout=layout)
    return iplot(fig, filename=title)

def plot_two_learning_curve(estimator,estimator_sm, title, X, y, cv,name):
    learning_curve_traces = plot_learning_curve(estimator.best_estimator_,"Learning curve of {} ".format(name), X, y, cv=cv,visible=True)
    learning_curve_sm_traces = plot_learning_curve(estimator_sm.best_estimator_,"Learning curve of {} with SMOTE".format(name), X, y, cv=cv,visible=False)
    steps = []
    step1 = dict(
        method='restyle',
        args=['visible',[False]*12],
        label = 'No SMOTE',
    )
    step1['args'][1][0] =True
    step1['args'][1][1] =True
    step1['args'][1][2] =True
    step1['args'][1][3] =True
    step1['args'][1][4] =True
    step1['args'][1][5] =True
    steps.append(step1)

    step2 = dict(
        method='restyle',
        args=['visible',[False]*12],
        label = 'SMOTE',
    )
    step2['args'][1][6] =True
    step2['args'][1][7] =True
    step2['args'][1][8] =True
    step2['args'][1][9] =True
    step2['args'][1][10] =True
    step2['args'][1][11] =True

    steps.append(step2)

    sliders = [dict(
    active = 0,
    currentvalue = dict(
        prefix = "{} ".format(name),
        xanchor= 'center',
    ),
    pad = {"t": 50},
    steps = steps,
    )]

    layout = dict(
        sliders=sliders,
        title=title,
        autosize=True,
        yaxis=dict(
            title='F1 Score',
        ),
        xaxis=dict(
            title="#Training samples",
        ),
        legend=dict(
            x=0.8,
            y=0,
        ),
    )
    fig = dict(data=learning_curve_traces+learning_curve_sm_traces,layout=layout)

    iplot(fig,filename='Learing curves')
    
    

In [973]:
#Split data in training and testing set

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3,stratify=y, random_state=SEED)
X_train_tree, X_test_tree, y_train_tree, y_test_tree = train_test_split(X_tree, y, test_size=0.3,stratify=y,random_state=SEED)
oversampling = SMOTE(random_state=SEED,k_neighbors=3)

kf = StratifiedKFold(n_splits=5, random_state=SEED,shuffle=True)

#Define parameters for models
parameters_knn = {'clf__n_neighbors':[3,5,7],'clf__weights':['uniform','distance'],'clf__p':[1,2,3]}
parameters_lr = {'clf__C':[0.001,0.01,0.1,1,10,100,1000],'clf__class_weight':['balanced']}
parameters_svm = {'clf__gamma':[0.001,0.01,0.1,1,10,100,1000],'clf__C':[0.001,0.01,0.1,1,10,100,1000],'clf__class_weight':['balanced'],'clf__kernel':['linear','rbf']}
parameters_decision_tree = {"clf__criterion":["gini","entropy"],'clf__max_depth':[2,4,8,16],'clf__min_samples_split':[2,4,8,10],'clf__class_weight':['balanced']}
parameters_random_forest = {"clf__criterion":["gini","entropy"],'clf__max_depth':[2,4,8,16],'clf__min_samples_split':[2,4,8,10],'clf__max_features':[2,3,4,5],'clf__n_estimators':[5,10,20],'clf__class_weight':['balanced']}

def trainingPipeline(name,clf, params, X_train, y_train,X_test,y_test, cv):

    pipeline = Pipeline([('clf', clf)])
    pipeline_sm = Pipeline([('oversapling', oversampling), ('clf', clf)])
    #No SMOTE
    gs = GridSearchCV(pipeline, params, cv=cv, n_jobs=-1, scoring='f1')
    gs.fit(X_train, y_train)
    print('The best parameters of {} model are: '.format(name),gs.best_params_)
    print('Best f1_score: {}'.format(gs.best_score_))
    gs_score = gs.score(X_test, y_test)
  
    y_pred = gs.predict(X_test)
    #confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] # normalize the confusion matrix
    cm_df = pd.DataFrame(cm.round(3), index=["No disease", "Parkinson"], columns=["predicted no disease", "predicted Parkinson"])
    report = classification_report(y_test, y_pred,output_dict=True)



    #SMOTE
    gs_sm = GridSearchCV(pipeline_sm, params, cv=cv, n_jobs=-1, scoring='f1')
    gs_sm.fit(X_train, y_train)
    print('The best parameters of {} model with SMOTE are: '.format(name),gs_sm.best_params_)
    print('Best f1_score: {}'.format(gs_sm.best_score_))
    gs_sm_score = gs_sm.score(X_test, y_test)
    y_pred_sm = gs_sm.predict(X_test)
    #confusion matrix
    cm_sm = confusion_matrix(y_test, y_pred_sm)
    cm_sm = cm_sm.astype('float') / cm_sm.sum(axis=1)[:, np.newaxis] # normalize the confusion matrix
    cm_sm_df = pd.DataFrame(cm_sm.round(3), index=["No disease", "Parkinson"], columns=["predicted no disease", "predicted Parkinson"])
    report_sm = classification_report(y_test, y_pred_sm,output_dict=True)
    


    return gs,gs_sm,report,report_sm,cm_df,cm_sm_df

   





## Synthetic Minority Over-sampling Technique

In this dataset we have the problem of **class imbalance**.In the exploration and visualization of the data it emerged that 24% of the available examples are attributable to the positive class. Models who are trained with this data have less chance of learning the patterns of the Parkinson class. In order to overcome this, the Syntetic oversampling technique was proposed.The minority class is over-sampled by taking each minority class sample and introducing synthetic examples along the line segments joining any/all of the k minority class nearest neighbors.
In this work we want to compare the performances obtained by the various algorithms with and without SMOTE.
The SMOTE implementation provided by the *imblearn* library was used.In the following graph is show the distribution of TRAINING DATA the application of SMOTE.



In [942]:
X_res, y_res = sm.fit_resample(X_train, y_train)


fig = px.scatter(x=X_res[:,0],y=X_res[:,1], color=y_res)
fig.show()

label_dict = {0: "No disease", 1: "Parkinson"}

yi = y_res.value_counts()

data_chart = [go.Bar(x=[label_dict[x] for x in yi.index], y=yi.values, marker = dict(color = colors[:len(yi.index)]))]
layout = go.Layout(
    title='Class label distribution',
    autosize=False,
    width=400,
    height=400,
    yaxis=dict(
        title='#samples',
    ),
)
fig = go.Figure(data=data_chart, layout=layout)
iplot(fig, filename='basic-bar3')

## Logistic Regression

This machine learning algorithm aims to model the condition probability like:

- $ \mathbb{P}(\mathbf{Y} = 1 /\mathbf{X}) = \frac{exp(\beta_{0}+\sum \beta_{i}\mathbf{X})}{1 + exp(\beta_{0}+\sum \beta_{i}\mathbf{X})} $

where  $\beta_{0},...,\beta_{n}$ are the parameters that you want to estimate.You need to learn a linear function $f(x_{1},....,x_{n}) = \beta_{0} + x_{1}\beta_{1} +...+x_{n}\beta_{n} $.To output a probability value ,this function is the input of an **activation function** called $ Sigmoid(x) = \frac{exp(x)}{1 + exp(x)} $.
The idea is to maximize  $ \mathbb{P}(\mathbf{Y} = 1 /\mathbf{X}) $ by considering all the training data.
To do that it uses **maximum likelihood** to estimate the parameters :

- $ \mathit{l}(\beta_{0},...,\beta_{i}) = \prod_{i:y_{i}==1} \mathbb{P}(\mathbf{Y} = 1 /\mathbf{X})\prod_{i:y_{i}==0}  1 - \mathbb{P}(\mathbf{Y} = 1 /\mathbf{X}) $.

For the objective function the algorithm uses the **log-likelihood** for numerical reasons that is maximized by using the **gradient-ascent** algorithm.To disadvantage too complex solutions that can lead to overfitting, a term equal to the norm of the weight vector has been added to the object function $ \mathbb{\beta} = \langle \beta_{0},...,\beta_{i} \rangle $.


The hyperparameter that we need to set is $ C $ that weight the regularization term.Smaller values specify stronger regularization.The possible choices for parameter $ C $ are:

<!-- -->  | <!-- -->| <!-- --> | <!-- --> | <!-- --> | <!-- -->| <!-- -->| <!-- -->
------ | ------| ------ | ------ | ------ | ------| ------| ------
$ C $      | $ 10^{-3} $| $ 10^{-2} $ | $ 10^{-1} $ | 1 | $ 10 $| $ 10^{2} $| $ 10^{3} $



In [943]:
lr = LogisticRegression()
glr,glr_sm,report,report_sm,cm_df,cm_sm_df = trainingPipeline('Logistic Regression',lr, parameters_lr, X_train, y_train,X_test,y_test, kf)

The best parameters of Logistic Regression model are:  {'clf__C': 1, 'clf__class_weight': 'balanced'}
Best f1_score: 0.5818922305764411
The best parameters of Logistic Regression model with SMOTE are:  {'clf__C': 1, 'clf__class_weight': 'balanced'}
Best f1_score: 0.5540049218067794


### Classification report Logistic Regression no SMOTE

In [944]:
df = pd.DataFrame(report).transpose()
df

,precision,recall,f1-score,support
0,0.971429,0.772727,0.860759,44.000000
1,0.583333,0.933333,0.717949,15.000000
accuracy,0.813559,0.813559,0.813559,0.813559
macro avg,0.777381,0.853030,0.789354,59.000000
weighted avg,0.872760,0.813559,0.824452,59.000000


### Classification report Logistic Regression with SMOTE

In [945]:
pd.DataFrame(report_sm).transpose()

,precision,recall,f1-score,support
0,0.944444,0.772727,0.850000,44.00000
1,0.565217,0.866667,0.684211,15.00000
accuracy,0.796610,0.796610,0.796610,0.79661
macro avg,0.754831,0.819697,0.767105,59.00000
weighted avg,0.848031,0.796610,0.807850,59.00000


### Confusion matrix Logistic Regression no SMOTE

In [918]:
cm_df

,predicted no disease,predicted Parkinson
No disease,0.773,0.227
Parkinson,0.067,0.933


### Confusion matrix Logistic Regression with SMOTE

In [946]:
cm_sm_df

,predicted no disease,predicted Parkinson
No disease,0.773,0.227
Parkinson,0.133,0.867


In [920]:
plot_two_learning_curve(glr,glr_sm,'Logistic Regression', X_train, y_train,kf,'Logistic Regression')

The model achieves satisfactory performance. The use of SMOTE does not lead to significant advantages.

# Support Vector Machines

The **Support Vector Machines** is a machine learning algorithm that aim to find the best hyperplane, the hyperplane with maximum distance between the two classes of data.It’s an optimization algorithm and differently from logistic regression only a subset of training data is relevant for the solution of the problem, the so called **support vectors**.These points are on the margin.

The **hard-margin** problem of SVM is :

$ maximize_{\beta_{0},\beta_{1},...,\beta_{p}} M $

$ \ \ subject \ to \sum^{p}_{j=1}{\beta^{2}_{j} = 1} \ \ and \ \ y_{i}\left[ \beta_{0} + \beta_{1}x_{i1}+...+\beta_{p}x_{ip}     \right] \ge M $ for all i = 1,...,N.

The formula of an hyperplane:

$  \beta_{0}+\beta_{1}x_{1}+...+\beta_{p}x_{p} = 0 $

This formulation outputs an hyperplane that perfectly separate the two classes.In major part of case it is needed to relax this constrain and to allow some errors.This is due to the fact that in some cases the data are not perfectly separable linearly and if they were it is preferable to have solutions that accept some errors for reasons related to overfitting. The
impact of this errors to the solution is measured by some slack variables $\zeta_{i} $.

The new **soft-margin** definition of the problem is:

$ maximize_{\beta_{0},\beta_{1},...,\beta_{p}} M $

$\ subject \ to \sum^{p}_{j=1}{\beta^{2}j} \ \ and \ \ y_{i} \left[ \beta_{0}+\beta_{1}x_{1}+...+\beta_{p}x_{p}  \right] \ge M\left(1 - \zeta_{i} \right) ,\ \ \zeta_{i} \ge 0, \ \ \sum^{n}_{i = 1} \zeta_{i} \le C $


This formulation add to the problem an hyperparameter to optimize,  $ C $. An higher
value of $ C $ increase the weight of errors and so the final solution will allow less errors.A lower value decrease the weight of errors and so the final solution will permit more errors.

In other situations data are not linearly separable.SVM is a linear classifier and to make it non-linear a solution is to find a transformation to project data in an higher dimensional space in which data are linearly separable.Instead of defining a transformation on the data and applying it, you can take advantage of the **kernel trick**.
A **kernel** is a function that accept two inputs and return the inner product of these two points in the features space given by the transformation $ \phi $:

- $ K(x_{i},x_{j}) =\langle \phi(x_{i}),\phi(x_{j}) \rangle $

There are a lot of kernel functions that can solve different problems of different data distributions, in this work we have used **linear kernel** and **RBF kernel**:

- RBF kernel: $K\left(x_{i},x_{j} \right) = exp \left(-\gamma \|x_{i} - x_{j}\| \right) $
- linear kernel: $ K\left(x_{i},x_{j} \right) = \langle x_{i},x_{j}\rangle $

The **RBF kernel** introduces a new parameter $\gamma$ that weights  the sensitivity of the difference between two examples.

We need to find the best $C$,$\gamma$ and kernel.The possible choices are:


<!-- -->  | <!-- --> |
------ | ------ |
$ C $      | $ 10^{-3} $, $ 10^{-2} $ , $ 10^{-1} $ , 1 , $ 10 $ , $ 10^{2} $ , $ 10^{3} $ |
$ \gamma $ | $ 10^{-3} $, $ 10^{-2} $ , $ 10^{-1} $ , 1 , $ 10 $ , $ 10^{2} $ , $ 10^{3} $ |
kernel   | linear,rbf |


In [947]:
c_svm = svm.SVC(random_state=SEED, probability=True)
gsvm,gsvm_sm,report,report_sm,cm_df,cm_sm_df = trainingPipeline('Support Vector Machines',c_svm, parameters_svm, X_train, y_train,X_test,y_test, kf)

The best parameters of Support Vector Machines model are:  {'clf__C': 10, 'clf__class_weight': 'balanced', 'clf__gamma': 0.1, 'clf__kernel': 'rbf'}
Best f1_score: 0.6960389610389611
The best parameters of Support Vector Machines model with SMOTE are:  {'clf__C': 1, 'clf__class_weight': 'balanced', 'clf__gamma': 0.1, 'clf__kernel': 'rbf'}
Best f1_score: 0.7201515151515151


### Classification report SVM no SMOTE

In [948]:
pd.DataFrame(report).transpose()

,precision,recall,f1-score,support
0,1.000000,0.863636,0.926829,44.000000
1,0.714286,1.000000,0.833333,15.000000
accuracy,0.898305,0.898305,0.898305,0.898305
macro avg,0.857143,0.931818,0.880081,59.000000
weighted avg,0.927361,0.898305,0.903059,59.000000


### Classification report SVM with SMOTE

In [949]:
pd.DataFrame(report_sm).transpose()

,precision,recall,f1-score,support
0,1.000000,0.818182,0.900000,44.000000
1,0.652174,1.000000,0.789474,15.000000
accuracy,0.864407,0.864407,0.864407,0.864407
macro avg,0.826087,0.909091,0.844737,59.000000
weighted avg,0.911570,0.864407,0.871900,59.000000


### Confusion matrix SVM no SMOTE

In [950]:
cm_df

,predicted no disease,predicted Parkinson
No disease,0.864,0.136
Parkinson,0.000,1.000


### Confusion matrix SVM with SMOTE

In [951]:
cm_sm_df

,predicted no disease,predicted Parkinson
No disease,0.818,0.182
Parkinson,0.000,1.000


In [952]:
plot_two_learning_curve(gsvm,gsvm_sm,'Support Vector Machines', X_train, y_train,kf,'Support Vector Machines')

This model achieves excellent performance. He is able to perfectly classify all examples of the Parkinson class.
The learning curve shows a good learning process and a good degree of generalization.

# K Nearest Neighbours

**K - Nearest Neighbors** is a machine learning algorithm that assume that the behavior of a data point is similar to the behavior of its nearest neighbors.To classify a data point it considers (given a distance/similarity measure) the label of the k - closest points in the space and assign.The parameters to optimize are:

- **k**: number of neighbours that the algorithm will use.
- **weight**: this parameter can be equal to *uniform* if all the neibours are are weighted equally or *distance* if you want to weight the neighbours by the inverse of the distance.
- **p** : is the power parameter of the Minkowski distance.The general formula is:$\\ \\ \ \ \ \  minkowski \ \ distance = \left( \sum_{i = 1}^{n} \left| x_{i} - y_{i} \right|^{\frac{1}{p}} \right)^{p} \\$
  If  $ p = 1 $ you have the so called  taxi distance ,otherwise if $ p = 2 $ you have the euclidean distance.

The possible choices for these parameters are:


<!-- -->  | <!-- -->|
------ | ------|
$ \mathbf{k} $ |     3,5,7
 **weight**  | distance , uniform
$ \mathbf{p} $ | 1,2,3

In [953]:
knn = KNeighborsClassifier()
gknn,gknn_sm,report,report_sm,cm_df,cm_sm_df = trainingPipeline('k-nearest neighbours',knn, parameters_knn, X_train, y_train,X_test,y_test, kf)

The best parameters of k-nearest neighbours model are:  {'clf__n_neighbors': 3, 'clf__p': 1, 'clf__weights': 'distance'}
Best f1_score: 0.6657142857142857
The best parameters of k-nearest neighbours model with SMOTE are:  {'clf__n_neighbors': 5, 'clf__p': 2, 'clf__weights': 'distance'}
Best f1_score: 0.7794117647058822


### Classification report KNN no SMOTE

In [960]:
pd.DataFrame(report).transpose()

,precision,recall,f1-score,support
0,1.000000,0.977273,0.988506,44.000000
1,0.937500,1.000000,0.967742,15.000000
accuracy,0.983051,0.983051,0.983051,0.983051
macro avg,0.968750,0.988636,0.978124,59.000000
weighted avg,0.984110,0.983051,0.983227,59.000000


### Classification report KNN with SMOTE

In [961]:
pd.DataFrame(report_sm).transpose()

,precision,recall,f1-score,support
0,1.000000,0.795455,0.886076,44.000000
1,0.625000,1.000000,0.769231,15.000000
accuracy,0.847458,0.847458,0.847458,0.847458
macro avg,0.812500,0.897727,0.827653,59.000000
weighted avg,0.904661,0.847458,0.856370,59.000000


### Confusion matrix KNN with SMOTE

In [962]:
cm_df

,predicted no disease,predicted Parkinson
No disease,0.977,0.023
Parkinson,0.000,1.000


### Confusion matrix KNN with SMOTE

In [963]:
cm_sm_df

,predicted no disease,predicted Parkinson
No disease,0.795,0.205
Parkinson,0.000,1.000


In [959]:
plot_two_learning_curve(gknn,gknn_sm,'KNN', X_train, y_train,kf,'KNN')

This classifier is able to obtain excellent results in the prediction of the Parkinson's class.TUsing smote brings no useful changes. There is an increase in the prediction of false positives.

# Decision Tree

The **decision tree** is a machine learning algorithm that,given the training set, at each step select one or more features and split the input data considering the values taken by these examples in the selected features.
This is done until the tree reach a desired **max_depth**, the splits contains only examples belonging to a single class or the split contains a number of splits lower than **min_samples_split**.The process of features selection at each step can be done randomly or by considering some metrics.
At each iteration you want to finf the best feature or subset of features which allows you to better separate the examples of the different classes.
You can measure how well a split is by using these two metrics:

- **gini index**: $ G = \sum^{K}_{k = 1} p_{mk}\left(1 - p_{mk} \right)$ ,where $ p_{mk} $ represents the proportion of training observations in the split $m$th that belongs to $k$th class.It is a measure of purity of a node.A small value means that the major part of examples in the node ara associate to the same class.  
- **cross  entropy:** $ D = -\sum^{K}_{k = 1 } p_{mk}log\left(p_{mk}\right) $


The main advantages related to decision trees are related to their simplicity and their comprehensibility.The main disadvantage is related to the fact that their performances are generally lower than other algorithms.

The hyperparameters that we need to select are:

- **criterion**: Gini index or cross entropy
- **max_deep**: The max deep that the final tree can reach.
- **min_saples_split**: The minimum number of samples required to split an internal node.

The grid of possible selection for the hyperparameters is:

<!-- -->  | <!-- -->
------ | ------ |
 **criterion** | gini,entropy |
 **max_deep**  | 2,4,8,16 |
 **min_sample_split**  | 2,4,8,10 |
 
 
Since the decision tree algorithm is able to select the most useful features, it makes no sense to use the preprocessing used for the previous algorithms, so we resume the dataset before the feature selection and the pca.

In [964]:
tree = DecisionTreeClassifier(random_state=SEED)
gtree,gtree_sm,report,report_sm,cm_df,cm_sm_df = trainingPipeline('Decision tree',tree, parameters_decision_tree, X_train_tree, y_train_tree,X_test_tree,y_test_tree, kf)

The best parameters of Decision tree model are:  {'clf__class_weight': 'balanced', 'clf__criterion': 'gini', 'clf__max_depth': 4, 'clf__min_samples_split': 12}
Best f1_score: 0.6667998667998668
The best parameters of Decision tree model with SMOTE are:  {'clf__class_weight': 'balanced', 'clf__criterion': 'entropy', 'clf__max_depth': 3, 'clf__min_samples_split': 2}
Best f1_score: 0.6725490196078432


In [965]:
def plot_feature_importance(feature_importance, title):
    trace1 = go.Bar(
        x=feature_importance[:, 0],
        y=feature_importance[:, 1],
        marker = dict(color = colors[0]),
        name='feature importance'
    )
    data = [trace1]
    '''
    layout = go.Layout(
        title=title,
        autosize=True,
        margin=go.layout.Margin(l=50, r=100, b=150),
        xaxis=dict(
            title='feature',
            tickangle=30
        ),
        yaxis=dict(
            title='feature importance',
            automargin=True,
        ),
    )
    '''
    fig = go.Figure(data=data)
    return iplot(fig, filename=title)


### Classification report Decision tree no SMOTE

In [966]:
pd.DataFrame(report).transpose()

,precision,recall,f1-score,support
0,0.891304,0.931818,0.911111,44.000000
1,0.769231,0.666667,0.714286,15.000000
accuracy,0.864407,0.864407,0.864407,0.864407
macro avg,0.830268,0.799242,0.812698,59.000000
weighted avg,0.860269,0.864407,0.861071,59.000000


### Classification report Decision tree with SMOTE

In [967]:
pd.DataFrame(report_sm).transpose()

,precision,recall,f1-score,support
0,0.966667,0.659091,0.783784,44.000000
1,0.482759,0.933333,0.636364,15.000000
accuracy,0.728814,0.728814,0.728814,0.728814
macro avg,0.724713,0.796212,0.710074,59.000000
weighted avg,0.843639,0.728814,0.746304,59.000000


### Confusion matrix Decision tree no SMOTE

In [968]:
cm_df

,predicted no disease,predicted Parkinson
No disease,0.932,0.068
Parkinson,0.333,0.667


### Confusion matrix Decision tree with SMOTE

In [969]:
cm_sm_df

,predicted no disease,predicted Parkinson
No disease,0.659,0.341
Parkinson,0.067,0.933


### Features importance

The Decision Tree class of sklearn also computes an importance value for each feature. This is done by weighting the decrease of impurity at each split by the probability of reaching this node for each node which split involves the feature of interest. Then these weighted decreases of impurity are summed up for each feature and this gives the feature importance.

In [970]:

features_importance_tree = sorted(zip(X_train_tree.columns, gtree.best_estimator_.named_steps['clf'].feature_importances_), key=lambda x: x[1], reverse=True)
features_importance_tree = [f for f in features_importance_tree if f[1] > 0]
features_importance_tree_sm = sorted(zip(X_train_tree.columns, gtree_sm.best_estimator_.named_steps['clf'].feature_importances_), key=lambda x: x[1], reverse=True)
features_importance_tree_sm = [f for f in features_importance_tree_sm if f[1] > 0]
step_name =['Decision Tree','Decision Tree with SMOTE']
plot_two_feature_importance(features_importance_tree,features_importance_tree_sm,step_name)

In [971]:
plot_two_learning_curve(gtree,gtree_sm,'Decision Tree', X_train_tree, y_train_tree,kf,'Decision Tree')

The use of smote in training leads to a significant increase in positive predictions but at the cost of an increase in false positives.

# Random Forest

The **Random Forest** is an example of **bagging** technique applied to Decision Trees.The **bagging** technique aims to improve the performance of a weak classifier by training more than one weak predictors by using different subsets of training set.
The final prediction is given by the vote of each individual predictor. The class with the most votes will be the class predicted by the model.This idea is given to the fact that given a set of observation $ Z_{1},...,Z_{n} $ each with variance $ \sigma^{2} $ the variace of the mean of the observations $ \overline{Z} $ is given by $ \sigma^{2}/n $. **Random Forest** trains a set of Decision Trees by using different boostrapped subsets and each tree can use only a random subset $ m $ of features to performs the splits.
The hyperparameters that you need to set are the same of Decision Tree plus the **max_features** that defines the number of features that can be randomly sampled for a given tree and the number of estimators **n_estimators**.




<!-- -->  | <!-- -->
------ | ------ |
 **criterion** | gini,entropy |
 **max_deep**  | 2,4,8,16|
 **min_sample_split**  | 2,4,8,10 |
 **max_features** | 2,3,4,5 |
 **n_estimators** | 5,10,20 |




In [975]:
forest = RandomForestClassifier(random_state=SEED)
gforest,gforest_sm,report,report_sm,cm_df,cm_sm_df = trainingPipeline('Random Forest',forest, parameters_random_forest, X_train_tree, y_train_tree,X_test_tree,y_test_tree, kf)

The best parameters of Random Forest model are:  {'clf__class_weight': 'balanced', 'clf__criterion': 'gini', 'clf__max_depth': 4, 'clf__max_features': 5, 'clf__min_samples_split': 4, 'clf__n_estimators': 20}
Best f1_score: 0.7655677655677656
The best parameters of Random Forest model with SMOTE are:  {'clf__class_weight': 'balanced', 'clf__criterion': 'gini', 'clf__max_depth': 4, 'clf__max_features': 4, 'clf__min_samples_split': 2, 'clf__n_estimators': 10}
Best f1_score: 0.8218315018315018


### Classification report Random Forest

In [976]:
pd.DataFrame(report).transpose()

,precision,recall,f1-score,support
0,0.955556,0.977273,0.966292,44.000000
1,0.928571,0.866667,0.896552,15.000000
accuracy,0.949153,0.949153,0.949153,0.949153
macro avg,0.942063,0.921970,0.931422,59.000000
weighted avg,0.948695,0.949153,0.948562,59.000000


### Classification report Random Forest with SMOTE

In [977]:
pd.DataFrame(report_sm).transpose()

,precision,recall,f1-score,support
0,0.975000,0.886364,0.928571,44.000000
1,0.736842,0.933333,0.823529,15.000000
accuracy,0.898305,0.898305,0.898305,0.898305
macro avg,0.855921,0.909848,0.876050,59.000000
weighted avg,0.914451,0.898305,0.901866,59.000000


### Confusion matrix  Random Forest 

In [978]:
cm_df

,predicted no disease,predicted Parkinson
No disease,0.977,0.023
Parkinson,0.133,0.867


### Confusion matrix  Random Forest with SMOTE

In [979]:
cm_sm_df

,predicted no disease,predicted Parkinson
No disease,0.886,0.114
Parkinson,0.067,0.933


In [980]:
features_importance_tree = sorted(zip(X_train_tree.columns, gforest.best_estimator_.named_steps['clf'].feature_importances_), key=lambda x: x[1], reverse=True)
features_importance_tree = [f for f in features_importance_tree if f[1] > 0]
features_importance_tree_sm = sorted(zip(X_train_tree.columns, gforest_sm.best_estimator_.named_steps['clf'].feature_importances_), key=lambda x: x[1], reverse=True)
features_importance_tree_sm = [f for f in features_importance_tree_sm if f[1] > 0]
step_name =['Random Forest','Random Forest with SMOTE']
plot_two_feature_importance(features_importance_tree,features_importance_tree_sm,step_name)

In [981]:
plot_two_learning_curve(gforest,gforest_sm,'Random Forest', X_train_tree, y_train_tree,kf,'Random Forest')

In the learning curve a slight overfitting can be observed, however the final performances are extremely positive.

# Comparison

In this section we compare the results of each ML algorithm (with or without SMOTE) by using the following metrics: **accuracy**,**precision**,**recall**,**AUC**,**f1**.
We plot the ROC curves of all the trained models.

In [982]:
def plot_roc_curve(classifiers,classifiers_tree, legend, title, X_test,X_test_tree, y_test):
    trace1 = go.Scatter(
        x=[0, 1],
        y=[0, 1],
        showlegend=False,
        mode="lines",
        name="",
        line = dict(
            color = colors[0],
        ),
    )
    data = [trace1]
    aucs = []

    for clf, string, c in zip(classifiers, legend, colors):
     
  
        y_test_roc = np.array([([0, 1] if y else [1, 0]) for y in y_test])
        if string in classifiers_tree:
            y_score = clf.predict_proba(X_test_tree)
        else:
            y_score = clf.predict_proba(X_test)
            

        # Compute ROC curve and ROC area for each class
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(2):
            fpr[i], tpr[i], _ = roc_curve(y_test_roc[:, i], y_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])

        # Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(y_test_roc.ravel(), y_score.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
        aucs.append(roc_auc['micro'])

        trace = go.Scatter(
            x=fpr['micro'],
            y=tpr['micro'],
            showlegend=True,
            mode="lines",
            name=string + " (area = %0.2f)" % roc_auc['micro'],
            hoverlabel = dict(
                namelength=30
            ),
            line = dict(
                color = c,
            ),
        )
        data.append(trace)

    layout = go.Layout(
        title=title,
        autosize=False,
        width=550,
        height=550,
        yaxis=dict(
            title='True Positive Rate',
        ),
        xaxis=dict(
            title="False Positive Rate",
        ),
        legend=dict(
            x=0.4,
            y=0.06,
        ),
    )
    fig = go.Figure(data=data, layout=layout)
    return aucs, iplot(fig, filename=title)

In [983]:


classifiers = [gforest,gforest_sm,glr,glr_sm, gknn,gknn_sm, gsvm,gsvm_sm,gtree,gtree_sm]

classifier_names = ["Random Forest","Random Forest with SMOTE","Logistic Regression","Logistic Regression with SMOTE", "KNN","KNN with SMOTE", "SVM","SVM with SMOTE","Decision Tree","Decision Tree with SMOTE" ]
classifier_names_tree =["Decision Tree","Decision Tree with SMOTE" ,"Random Forest","Random Forest with SMOTE"]

auc_scores, roc_plot = plot_roc_curve(classifiers,classifier_names_tree, classifier_names, "ROC curve", X_test, X_test_tree, y_test)

In [984]:
accs = []
recalls = []
precision = []
results_table = pd.DataFrame(columns=["accuracy", "precision", "recall", "f1", "auc"])
for (i, clf), name, auc in zip(enumerate(classifiers), classifier_names, auc_scores):
    if name in classifier_names_tree:
        y_pred = clf.predict(X_test_tree)
    else:
        y_pred = clf.predict(X_test)
        
    row = []
    row.append(accuracy_score(y_test, y_pred))
    row.append(precision_score(y_test, y_pred))
    row.append(recall_score(y_test, y_pred))
    row.append(f1_score(y_test, y_pred))
    row.append(auc)
    row = ["%.3f" % r for r in row]
    results_table.loc[name] = row

In [985]:
results_table

,accuracy,precision,recall,f1,auc
Random Forest,0.949,0.929,0.867,0.897,0.985
Random Forest with SMOTE,0.898,0.737,0.933,0.824,0.958
Logistic Regression,0.814,0.583,0.933,0.718,0.915
Logistic Regression with SMOTE,0.797,0.565,0.867,0.684,0.912
KNN,0.983,0.938,1.000,0.968,0.979
KNN with SMOTE,0.847,0.625,1.000,0.769,0.940
SVM,0.898,0.714,1.000,0.833,0.997
SVM with SMOTE,0.864,0.652,1.000,0.789,0.976
Decision Tree,0.864,0.769,0.667,0.714,0.905
Decision Tree with SMOTE,0.729,0.483,0.933,0.636,0.874


Considering the auc alone, the SVM is the best model, followed by the random forest. In general, for all metrics, the models obtain excellent scores in all the metrics used. The use of SMOTE in general does not lead to significant advantages. A behavior observed in many models when using oversampling is the increase in positive predictions associated with an increase in false positives.

# Conclusion

In this analysis, we tried to distinguish among all the voice samples those associated with patients with Parkinson's. The dataset is characterized by a significant unbalance of classes in favor of No Disease class. For this reason it was decided to use the SMOTE oversampling technique, comparing it with a standard data analysis. Already in the first quantitative views of the data a substantial difference in values emerged between the Parkinson class and the No Disease cless.For the preprocessing it was decided to combine a features selection technique based on the correlation between features with a dimensionality reduction technique such as PCA. Finally, machine learning algorithms were used, the results of which confirmed the expectations.